### Downloading hadoop

In [1]:
! wget http://ftp.unicamp.br/pub/apache/hadoop/common/hadoop-2.9.2/hadoop-2.9.2.tar.gz -q --show-progress

hadoop-2.9.2.tar.gz 100%[===================>] 349.47M  18.9MB/s    in 19s     


### Decompressing

Optional: removing tar file

In [2]:
!tar -xvf hadoop-2.9.2.tar.gz >/dev/null 

### Checking $JAVA_HOME path

In [3]:
!dirname $(dirname $(readlink -f $(which javac)))

/usr/lib/jvm/java-8-openjdk-amd64


### Setting envvar $JAVA_HOME

In [4]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64

env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64


In [5]:
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " > .bashrc

### Checking envvar

In [6]:
%env JAVA_HOME

'/usr/lib/jvm/java-8-openjdk-amd64'

### Starting sshd server

Check postBuild and sshd_config files for more details

In [7]:
!/usr/sbin/sshd -f resources/configs/ssh/sshd_config 

### Adding to know hosts by sshing (avoiding yes/no host confirmation)

In [8]:
!ssh -o "StrictHostKeyChecking no" $USER@localhost -p 9000 -C "exit" 
!ssh -o "StrictHostKeyChecking no" $USER@0.0.0.0   -p 9000 -C "exit"

In [ ]:
! cp resources/configs/hadoop/* hadoop-2.9.2/etc/hadoop/

In [25]:
!./hadoop-2.9.2/bin/hdfs namenode -format -force -nonInteractive

In [21]:
%env HADOOP_SSH_OPTS -p 9000 

env: HADOOP_SSH_OPTS=-p 9000


In [22]:
! ./hadoop-2.9.2/sbin/stop-dfs.sh
! ./hadoop-2.9.2/sbin/start-dfs.sh

Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: namenode did not stop gracefully after 5 seconds: killing with kill -9
localhost: stopping datanode
localhost: datanode did not stop gracefully after 5 seconds: killing with kill -9
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
0.0.0.0: secondarynamenode did not stop gracefully after 5 seconds: killing with kill -9
Starting namenodes on [localhost]
localhost: starting namenode, logging to /home/jovyan/hadoop-2.9.2/logs/hadoop-jovyan-namenode-jupyter-thedatasociety-2dmybinder-2dhadoop-2dqf72qknj.out
localhost: starting datanode, logging to /home/jovyan/hadoop-2.9.2/logs/hadoop-jovyan-datanode-jupyter-thedatasociety-2dmybinder-2dhadoop-2dqf72qknj.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /home/jovyan/hadoop-2.9.2/logs/hadoop-jovyan-secondarynamenode-jupyter-thedatasociety-2dmybinder-2dhadoop-2dqf72qknj.out


In [23]:
!./hadoop-2.9.2/bin/hdfs dfs -mkdir /user
!./hadoop-2.9.2/bin/hdfs dfs -mkdir /user/matheus

mkdir: Failed on local exception: org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length; Host Details : local host is: "jupyter-thedatasociety-2dmybinder-2dhadoop-2dqf72qknj/10.12.45.131"; destination host is: "localhost":9000; 
mkdir: Failed on local exception: org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length; Host Details : local host is: "jupyter-thedatasociety-2dmybinder-2dhadoop-2dqf72qknj/10.12.45.131"; destination host is: "localhost":9000; 


In [24]:
!./hadoop-2.9.2/bin/hdfs dfs -ls /

ls: Failed on local exception: org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length; Host Details : local host is: "jupyter-thedatasociety-2dmybinder-2dhadoop-2dqf72qknj/10.12.45.131"; destination host is: "localhost":9000; 
